## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv() ## Loading all the enviroment varaible

groq_api_key = os.getenv("GROQ_API_KEY")


In [3]:
from langchain_groq import ChatGroq
model = ChatGroq(model="Gemma2-9b-It", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7fb1546606e0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7fb154662a50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [ ]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, My name is Sadeeq and I and a Chief AI Engineer")])

AIMessage(content="Hi Sadeeq, it's nice to meet you! That's a very interesting role. \n\nWhat kind of projects are you working on as a Chief AI Engineer? I'm always curious to learn more about the applications of AI.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 23, 'total_tokens': 76, 'completion_time': 0.096363636, 'prompt_time': 0.002277652, 'queue_time': 0.23010892500000002, 'total_time': 0.098641288}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-5b56d49b-6198-4147-b4c9-f31804ba6f99-0', usage_metadata={'input_tokens': 23, 'output_tokens': 53, 'total_tokens': 76})

In [5]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, My name is Sadeeq and I and a Chief AI Engineer"),
        AIMessage(content="Hi Sadeeq, it's nice to meet you! That's a very interesting role. \n\nWhat kind of projects are you working on as a Chief AI Engineer? I'm always curious to learn more about the applications of AI.\n"),
        HumanMessage(content="Hey Whats my name and what do i do?")
    ]
)

AIMessage(content="You are Sadeeq, and you are a Chief AI Engineer!  😊  \n\nIs there anything else you'd like to know or discuss? I'm here to help.  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 94, 'total_tokens': 135, 'completion_time': 0.074545455, 'prompt_time': 0.004323057, 'queue_time': 0.231746731, 'total_time': 0.078868512}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-afbff481-dafe-49b5-a9ca-ea008b70d37d-0', usage_metadata={'input_tokens': 94, 'output_tokens': 41, 'total_tokens': 135})

### Message History
we can use a Message history class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. let's see how to use this!